## Exploratory Data Analysis

### Previewing Data, Checking Unique Fields, & Calculating General Statistics

In [3]:
import os
import pandas as pd
import numpy as np

In [7]:
PREPROCESSED_DATA_PATH = "../../data/preprocessed/"
PREPROCESSED_DATA_FILES = os.listdir(PREPROCESSED_DATA_PATH)

In [69]:
ALL_RACE_DATA_DF = pd.DataFrame()
for file in PREPROCESSED_DATA_FILES:
    if file == "drivers.csv":
        continue
    file_path = os.path.join(PREPROCESSED_DATA_PATH, file)
    current_year_df = pd.read_csv(file_path)
    ALL_RACE_DATA_DF = pd.concat([ALL_RACE_DATA_DF, current_year_df])

In [70]:
DRIVERS_DF = pd.read_csv(os.path.join(PREPROCESSED_DATA_PATH, "drivers.csv"))

#### Unique Constructor Teams (2018 - 2024)

First, I want to see how many teams are in my dataset to check naming convention and see how rebrandings were handled (e.g. "Force India" to "Racing Point" to "Aston Martin").

##### Constructors

In [71]:
TEAMS_DF = ALL_RACE_DATA_DF[["TeamId", "TeamName"]].drop_duplicates().sort_values(by="TeamId").reset_index(drop=True)
TEAMS_DF

,TeamId,TeamName
0,alfa,Alfa Romeo
1,alfa,Alfa Romeo Racing
2,alphatauri,AlphaTauri
3,alpine,Alpine
4,aston_martin,Aston Martin
5,ferrari,Ferrari
6,force_india,Force India
7,force_india,Racing Point
8,haas,Haas F1 Team
9,mclaren,McLaren


Based on this DataFrame, there seems to be some mismatch due to re-brandings and some naming inconsistencies.

<li>The sauber TeamId has 2 TeamName values: Sauber and Kick Sauber</li>
<li>The alfa TeamId has 2 TeamName values: Alfa Romeo and Alfa Romeo Racing</li>
<li>The Alfa Romeo and Kick Sauber teams are the same due to re-branding</li>
<li>The rebranding for the current RB team has multiple TeamId and TeamName values (toro_rosso - Toro Rosso, alphatauri - AlphaTauri, rb - RB)</li>
<li>The rebranding for the current Alpine team has multiple TeamId and TeamName values (renault - Renault, alpine - Alpine)</li>
<li>The rebranding for the current Aston Martin team has multiple TeamId and TeamName values (force_india - Force India, force_india - Racing Point, aston_martin - Aston Martin)</li>

In [72]:
def fix_team_data(df:pd.DataFrame):
    # Fix the Sauber and Alfa Romeo teams
    df["TeamId"] = df["TeamId"].replace("alfa", "sauber")
    df["TeamName"] = df["TeamName"].replace("Alfa Romeo Racing", "Kick Sauber")
    df["TeamName"] = df["TeamName"].replace("Alfa Romeo", "Kick Sauber")
    df["TeamName"] = df["TeamName"].replace("Sauber", "Kick Sauber")

    # Rename the old RB TeamIds and TeamNames with the current values
    df["TeamId"] = df["TeamId"].replace(["toro_rosso", "alphatauri"], ["rb", "rb"])
    df["TeamName"] = df["TeamName"].replace(["Toro Rosso", "AlphaTauri"], ["RB", "RB"])

    # Replace the Renault TeamId and TeamName with the current Alpine team's
    df["TeamId"] = df["TeamId"].replace("renault", "alpine")
    df["TeamName"] = df["TeamName"].replace("Renault", "Alpine")

    # Rename the old Aston Martin TeamIds and TeamNames with the current values
    df["TeamId"] = df["TeamId"].replace("force_india", "aston_martin")
    df["TeamName"] = df["TeamName"].replace(["Force India", "Racing Point"], ["Aston Martin", "Aston Martin"])

    return df.drop_duplicates().sort_values(by="TeamId").reset_index(drop=True)

TEAMS_DF = fix_team_data(TEAMS_DF)
TEAMS_DF

,TeamId,TeamName
0,alpine,Alpine
1,aston_martin,Aston Martin
2,ferrari,Ferrari
3,haas,Haas F1 Team
4,mclaren,McLaren
5,mercedes,Mercedes
6,racing_point,Aston Martin
7,rb,RB
8,red_bull,Red Bull Racing
9,sauber,Kick Sauber


This final DataFrame shows the 10 teams which is aligned with my knowedge and are currently on the grid as of the creation of this project in August and September 2025.

The corrections made in the cell above will be part of the data processing phase.

In [73]:
ALL_RACE_DATA_DF = fix_team_data(ALL_RACE_DATA_DF)

#### Broad Season Statistics (2018 - 2024)

To begin with the data, let's start with the most important stats: total points by each constructor team for each season.

In [74]:
POINTS_PER_SEASON_BY_TEAM_DF = ALL_RACE_DATA_DF[["Year", "TeamId", "Points"]].groupby(["Year", "TeamId"])["Points"].sum().reset_index()
POINTS_PER_SEASON_BY_TEAM_DF = POINTS_PER_SEASON_BY_TEAM_DF.sort_values(by=["Year", "Points"], ascending=[True, False]).reset_index(drop=True)

POINTS_PER_SEASON_BY_TEAM_DF.head(10)

,Year,TeamId,Points
0,2018,mercedes,655.0
1,2018,ferrari,571.0
2,2018,red_bull,419.0
3,2018,alpine,122.0
4,2018,aston_martin,111.0
5,2018,haas,93.0
6,2018,mclaren,62.0
7,2018,sauber,48.0
8,2018,rb,33.0
9,2018,williams,7.0


In [75]:
POINTS_PER_SEASON_BY_TEAM_DF.tail(10)

,Year,TeamId,Points
60,2024,mclaren,666.0
61,2024,ferrari,652.0
62,2024,red_bull,589.0
63,2024,mercedes,468.0
64,2024,aston_martin,94.0
65,2024,alpine,65.0
66,2024,haas,58.0
67,2024,rb,46.0
68,2024,williams,17.0
69,2024,sauber,4.0
